In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install PyMuPDF==1.23.26 matplotlib==3.8.3 numpy==1.26.4 pandas==2.2.1 Requests==2.31.0 sentence_transformers==2.5.1 spacy tqdm==4.66.2 transformers==4.38.2 accelerate bitsandbytes wheel


In [ ]:
cd drive/MyDrive/Prop_Bank/training_data

/content/drive/MyDrive/Prop_Bank/training_data


## RAG Search and Answer

Embeddings can be converted to any format

audio
images
text

Comparing embeddings to vector search, similarity search and semantic search
based on keywords I should get the sentences

In [ ]:
import random
import torch
import numpy as np
import pandas as pd

device = "cuda"

## loading the saved embedding file stored in string
embeddings_df = pd.read_csv("text_chunk_embeddings_df.csv")

## np array (string --> array)
embeddings_df["embedding"] = embeddings_df["embedding"].apply(lambda x:np.fromstring(x.strip("[]"), sep=" "))


##  load it to device and dtype should be float32
embeddings = torch.tensor(np.stack(embeddings_df["embedding"].tolist(), axis=0), dtype=torch.float32).to(device)

pages_and_chunks = embeddings_df.to_dict(orient="records")

embeddings_df

,pdf_name,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,Community-Engagement-Toolkit_Building_Purpose_...,1,Community Engagement Toolkit Building Purpose ...,158,21,39.50,"[-0.02657816, 0.0288857743, -0.0210008025, -0...."
1,Community-Engagement-Toolkit_Building_Purpose_...,2,This Toolkit was created in partnership by Ent...,1381,203,345.25,"[-0.0348760672, 0.118880965, -0.0503364652, -0..."
2,Community-Engagement-Toolkit_Building_Purpose_...,3,P14 P19 P23 P28 P31 P35 . U. S. Department of ...,1237,202,309.25,"[-0.021397816, 0.0299834404, -0.0297792666, -0..."
3,Community-Engagement-Toolkit_Building_Purpose_...,4,TOOLKIT INTRODUCTION US Department of Housing ...,1571,230,392.75,"[-0.0229254626, 0.0630211383, -0.0168476477, -..."
4,Community-Engagement-Toolkit_Building_Purpose_...,5,U. S. Department of Housing and Urban Developm...,1676,257,419.00,"[-0.0410287678, 0.0603567548, -0.0253214575, 0..."
...,...,...,...,...,...,...,...
290,guide_for_researchers.pdf,13,A Q U I C K - S TA R T G U I D E F O R R E S E...,1256,186,314.00,"[0.00325675332, 0.0540609173, -0.00894159824, ..."
291,guide_for_researchers.pdf,14,C O M M U N I T Y - E N GA G E D R E S E A R C...,952,179,238.00,"[-0.0222157054, 0.0353563726, -0.00829026569, ..."
292,guide_for_researchers.pdf,15,Publications and Online Resources The CTSI Com...,1415,194,353.75,"[0.00933027081, 0.0221390426, -0.0128135597, -..."
293,guide_for_researchers.pdf,15,From the Series: UCSF Clinical and Translation...,329,35,82.25,"[0.0190449208, -0.00812574476, -0.0181795917, ..."


## Creating a model

Note: to use dot product for comparison, we need to ensure that vector are of same sizes (768) and tensors/vectors are in same datatype

(Eg torch.float32)

In [ ]:

from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device=device)

Embedding model

Creating a search query and get back relevant passages from uploaded pdfs

1. Pass a string query
2. Turn the query string into embedding
3. Perform  a dot product or cosine similarity
4. Sort results from 3 descending order

     

In [ ]:
query = "community engaged research"
print(f"Query: {query}")


query_embedding = embedding_model.encode(query, convert_to_tensor=True).to(device, dtype=torch.float32)

from time import perf_counter as timer

embeddings = torch.tensor(embeddings).to(device, torch.float32)

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()


print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds")

top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: community engaged research
[INFO] Time taken to get scores on 295 embeddings: 0.00022 seconds


<ipython-input-12-b6292ac26fe1>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings = torch.tensor(embeddings).to(device, torch.float32)


torch.return_types.topk(
values=tensor([0.8392, 0.8302, 0.8023, 0.8002, 0.8001], device='cuda:0'),
indices=tensor([148, 167, 149, 278, 205], device='cuda:0'))

In [ ]:
pages_and_chunks[148]

{'pdf_name': 'CRC-Guidelines-May-12-2021.pdf',
 'page_number': 6,
 'sentence_chunk': '2 Community-based research (CBR) is in high demand. More and more, communities and academic researchers are partnering in order to learn about and address real-world issues. CBR is being used to: • Translate scientific knowledge into practice • Support organizing and movement building • Impact policy • Guide community and economic development • Foster learning and personal transformation • Build trust with communities harmed by past research •Improve organizations • Strengthen communities •Enrich our understanding of the world  Today, we are wrestling with deep-rooted inequities and global challenges that defy simple answers. CBR can be a powerful way to address these challenges by harness-ing our collective knowledge and resources. Unfortunate-ly, not everything that goes under the name “community-based research” lives up to the promise. More support is needed to help this work flourish.',
 'chunk_ch

In [ ]:

import textwrap

def print_wraps(text, wrap_length=100):
    wrap_text = textwrap.fill(text, wrap_length)
    print(wrap_text)

## video reference
We can see that concerns regarding which db to use and what is the threshold

if we replicate it is still faster with gpu (embeddings*100/1000)

In [ ]:
## now wrapping multiple iterables (eg: lists, tuples or strings)


query = "Community engaged research"
print(f"Query: '{query}'\n")
print("Results: ")


for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score: .4f}")
    print("Text:")

    print_wraps(pages_and_chunks[idx]["sentence_chunk"])
    print("\n")


Query: 'Community engaged research'

Results: 
Score:  0.8392
Text:
2 Community-based research (CBR) is in high demand. More and more, communities and academic
researchers are partnering in order to learn about and address real-world issues. CBR is being used
to: • Translate scientific knowledge into practice • Support organizing and movement building •
Impact policy • Guide community and economic development • Foster learning and personal
transformation • Build trust with communities harmed by past research •Improve organizations •
Strengthen communities •Enrich our understanding of the world  Today, we are wrestling with deep-
rooted inequities and global challenges that defy simple answers. CBR can be a powerful way to
address these challenges by harness-ing our collective knowledge and resources. Unfortunate-ly, not
everything that goes under the name “community-based research” lives up to the promise. More support
is needed to help this work flourish.


Score:  0.8302
Text:
Commun

# Similarity measures: dot product and cosine (example showcase)

similarity measures between vectors are dot product and cosine similarity

closer vecotrs will have higher scores and futher away will be lower

Vectors (direction of semantic search) amd magnitude (how long will it take)

## Functionizing it to for performing better function call

In [ ]:

# def retrieve_relevant_resources(query: str,
#                                 embeddings: torch.tensor,
#                                 model: SentenceTransformer = embedding_model,
#                                 n_resources_return: int=5,
#                                 print_time: bool=True):

#     query_embedding = model.encode(query, convert_to_tensor=True)

#     start_time = timer()
#     dot_scores = util.dot_score(query_embedding, embeddings)[0]
#     end_time = timer()

#     if print_time:
#         print(f"[INFO] Time taken to get scores on ({len(embeddings)}) embeddings: {end_time-start_time: .5f} seconds.")

#     scores, indices = torch.topk(input= dot_scores, k=n_resources_return)
#     return scores, indices


# def print_top_results_and_scores(query: str,
#                                  embeddings: torch.tensor,
#                                  pages_and_chunks: list[dict]=pages_and_chunks,
#                                  n_resources_return: int=5):

#     scores, ind = retrieve_relevant_resources(query=query,
#                                                   embeddings=embeddings,
#                                                   n_resources_return=n_resources_return)

#     for score, idx in zip(scores, ind):
#         print(f"Score: {score:.4f}")
#         print("Text:")
#         print_wraps(pages_and_chunks[idx]["sentence_chunk"])
#         print("\n")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from time import time


def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer = embedding_model,
                                n_resources_return: int = 5,
                                print_time: bool = True):

    query_embedding = model.encode(query, convert_to_tensor=True).cpu().numpy()

    start_time = time()
    similarity_scores = cosine_similarity(query_embedding.reshape(1, -1), embeddings.cpu().numpy())[0]
    end_time = time()

    if print_time:
        print(f"[INFO] Time taken to get scores on ({len(embeddings)}) embeddings: {end_time - start_time:.5f} seconds.")

    top_indices = similarity_scores.argsort()[-n_resources_return:][::-1]
    top_scores = similarity_scores[top_indices]

    return top_scores, top_indices

In [ ]:
def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict] = pages_and_chunks,
                                 n_resources_return: int = 5):
    scores, indices = retrieve_relevant_resources(query, embeddings, n_resources_return=n_resources_return)

    for score, idx in zip(scores, indices):
        print(f"Score: {score:.4f}")
        print("Text:")
        print_wraps(pages_and_chunks[idx]["sentence_chunk"])
        print("\n")

In [ ]:
query = "individual"

print_top_results_and_scores(query=query, embeddings=embeddings)

[INFO] Time taken to get scores on (295) embeddings:  0.00006 seconds.
Score: 0.2065
Text:
What internalized biases or assumptions have you come up against in this work? •What abilities or
capacities did you discover in yourself?


Score: 0.1975
Text:
Community Engagement Toolkit PART 1 - Introduction Page 13 U. S. Department of Housing and Urban
Development Picture this: A city holds a public hearing to invite feedback on a proposal for a large
mixed- use development in a larger neighborhood. The City informs the community about the meeting
through neighborhood online forums on Facebook. In theory, all community stakeholders are invited to
participate. In reality, not everyone has access to the information. Lower-income community members
may not have access to the Internet or forums in which the city is posting. New community members or
immigrant community members may not have been invited to participate in certain social groups, and
senior community members may not have Facebook at a

## Local LLM

We have take which model to load and parameters required for it
https://github.com/mrdbourke/simple-local-rag/blob/main/00-simple-local-rag.ipynb

there is table wise comparison on LLM diff size and diff levels of numerical precision

<b>Quantization is reducing the size of the model</b>

according to this tutorial it will Gemma 7b

VRAM is important

In [ ]:
## Checking GPU
import torch

gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes/ (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")


Available GPU memory: 40 GB


In [ ]:
## small snippet from https://github.com/mrdbourke/simple-local-rag/blob/main/video_notebooks/00-simple-local-rag-video.ipynb
## to check which model to use
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.

if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 40 | Recommend model: Gemma 7B in 4-bit or float16 precision.
use_quantization_config set to: False
model_id set to: google/gemma-7b-it


In [ ]:
!pip install -U bitsandbytes transformers accelerate

  Using cached transformers-4.48.0-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.48.0-py3-none-any.whl (9.7 MB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# template from hugging face gemma
# Use a pipeline as a high-level helper

from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="google/gemma-2-2b-it")
pipe(messages)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "I am Gemma, an open-weights AI assistant. I'm a large language model, trained"}]}]

In [ ]:
'''Create prompt template for instruction-tuned model
# Apply the chat template
'''

input_text = "What are the examples of community based research?"
print(f"Input text:\n{input_text}")

dialogue_template = [
    {"role": "user",
     "content": input_text}
]

prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False,
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")

Input text:
What are the examples of community based research?

Prompt (formatted):
<bos><start_of_turn>user
What are the examples of community based research?<end_of_turn>
<start_of_turn>model



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.utils import is_flash_attn_2_available

In [ ]:
def get_attention_implementation():
    if is_flash_attn_2_available() and torch.cuda.get_device_capability(0)[0] >= 8:
        return "flash_attention_2"
    else:
        return "sdpa"

In [ ]:
def get_quantization_config(use_quantization: bool):
    if use_quantization:
        return BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
    return None

In [ ]:
def load_tokenizer(model_id: str):
    return AutoTokenizer.from_pretrained(model_id)

In [ ]:
def load_model(model_id: str, quantization_config=None, attn_impl="sdpa"):
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        quantization_config=quantization_config,
        low_cpu_mem_usage=True,
        attn_implementation=attn_impl
    )
    if not quantization_config:
        model.to("cuda")
    return model

In [ ]:
model_id = "google/gemma-2-2b-it"
attn_impl = get_attention_implementation()
quant_config = get_quantization_config(use_quantization=True)

tokenizer = load_tokenizer(model_id)
llm_model = load_model(model_id, quantization_config=quant_config, attn_impl=attn_impl)

Using attention implementation: sdpa


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
%%time

# Tokenize the input text (turn it into numbers) and send it to the GPU
input_ids = tokenizer(prompt,
                      return_tensors="pt").to("cuda")

# Generate outputs from local LLM
outputs = llm_model.generate(**input_ids,
                             max_new_tokens=512)
print(f"Model output (tokens):\n{outputs[0]}\n")

Model output (tokens):
tensor([     2,      2,    106,  ..., 235248,    108,    107], device='cuda:0')

CPU times: user 17.1 s, sys: 24.9 ms, total: 17.1 s
Wall time: 17.1 s


In [ ]:
# Decode the output tokens to text
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Model output (decoded):
<bos><bos><start_of_turn>user
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.


 Example 1:
Query: what are the keywords for community based research?
Answer: Community-Based Research (CBR) often includes keywords that highlight collaboration, participation, and social impact. Common keywords are "community engagement," "stakeholder participation," "collaborative research," "co-creation," "grassroots," "community-driven," "social justice," "action research," "public health," "local knowledge," and "empowerment." These keywords emphasize inclusive approaches, focusing on addressing real-world community issues by actively involving the community throughout the research proces

# Augmenting prompt with context items

In [ ]:
query = "What are the factors for community based research?"

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)

scores, indices

[INFO] Time taken to get scores on (295) embeddings:  0.00010 seconds.


(tensor([0.7597, 0.7283, 0.7233, 0.7190, 0.7030], device='cuda:0'),
 tensor([167, 149, 283, 148, 278], device='cuda:0'))

In [ ]:
def prompt_formatter(query: str,
                     context_items : list[dict]) -> str:
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

\n Example 1:
Query: what are the keywords for community based research?
Answer: Community-Based Research (CBR) often includes keywords that highlight collaboration, participation, and social impact. Common keywords are "community engagement," "stakeholder participation," "collaborative research," "co-creation," "grassroots," "community-driven," "social justice," "action research," "public health," "local knowledge," and "empowerment." These keywords emphasize inclusive approaches, focusing on addressing real-world community issues by actively involving the community throughout the research process.

\n Example 2:
Query: what are the keywords for non community based research?
Answer: Non-Community-Based Research typically uses formal, academic, or institution-focused language, emphasizing theoretical frameworks, experimental studies, and scientific rigor. Keywords include "lab-based," "clinical trials," "randomized controlled experiments," "meta-analysis," "hypothesis testing," "generalization," "controlled environment," "literature review," "data-driven," and "empirical study." These studies often prioritize knowledge production without direct community involvement, aiming to produce results applicable to broader contexts rather than addressing specific community needs.

\n Example 3:
Query: How to differentiate them?
Answer: To differentiate, CBR focuses on solving community-specific problems through active community participation, involving keywords like “collaboration,” “action,” and “engagement.” In contrast, non-CBR is more theoretical or generalizable, often conducted in controlled environments, with keywords emphasizing “hypothesis,” “analysis,” and “experimentation.” CBR outputs typically address local, practical concerns, while non-CBR outputs aim for general scientific contributions, often without immediate local application. The presence of stakeholder involvement or a social impact focus is a key distinguishing factor for CBR.

\n Now use the following context items to answer the user query:
{context}
User query: {query}
Answer:
"""

    base_prompt = base_prompt.format(context=context, query=query)

    dialogue_template = [
        {"role": "user",
         "content": base_prompt}
    ]
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                           tokenize=False,
                                           add_generation_prompt=True)
    return prompt


query = "What are the factors for community based research?"
scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
context_items = [pages_and_chunks[idx] for idx in indices]
prompt = prompt_formatter(query=query,
                          context_items=context_items)
print(prompt)




[INFO] Time taken to get scores on (295) embeddings:  0.00006 seconds.
<bos><start_of_turn>user
Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.


 Example 1:
Query: what are the keywords for community based research?
Answer: Community-Based Research (CBR) often includes keywords that highlight collaboration, participation, and social impact. Common keywords are "community engagement," "stakeholder participation," "collaborative research," "co-creation," "grassroots," "community-driven," "social justice," "action research," "public health," "local knowledge," and "empowerment." These keywords emphasize inclusive approaches, focusing on addressing real-world community issues by actively involving th

In [ ]:
%%time

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")


outputs = llm_model.generate(**input_ids,
                             temperature=0.7, # from 0 to 1 and the lower the value, the more deterministic the text, the higher the value, the more creative
                             do_sample=True, # whether or not to use sampling, https://huyenchip.com/2024/01/16/sampling.html
                             max_new_tokens=256)

# Turn the output tokens into text
query_test = "Examples of community based research"
output_text = tokenizer.decode(outputs[0])
print(f"Query: {query_test}")
print(f"RAG answer:\m{output_text.replace(prompt, '')}")

Query: Examples of community based research
RAG answer:\m<bos>Factors for Community-Based Research:

* **Community Needs:** CBR prioritizes addressing community-specific problems and needs.
* **Community Involvement:**  Active participation of community members throughout the research process is crucial.
* **Collaboration and Power Sharing:** Partners share power and collaborate to develop and carry out the research.
* **Meaningful Questions:** Research questions are rooted in the community's concerns and experiences.
* **Solutions-Oriented Approach:** The focus is on implementing solutions to address community issues.
* **Expertise of Community Members:** Community members and professionals contribute their expertise and insights.
* **Trust Building:**  Establishing trust and strong relationships between partners is essential.
* **Long-Term Commitment:**  Long-term commitment from all parties is necessary.
* **Cultural Sensitivity:**  Respecting and understanding the cultural context 